In [1]:
import sys
sys.path.append(r"../")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pingouin as pg
#from main import create_dataset
import dtale
import os
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
from statsmodels.stats.contingency_tables import mcnemar
os.getcwd()

'C:\\Users\\nogur\\Documents\\DeppClinic\\source\\projects\\Liat graphs'

In [2]:
suicidal_ideation_time2 = [

    'c_ssrs_t_last_1_clin',
    'c_ssrs_t_last_2_clin',
    'c_ssrs_t_last_3_clin',
    'c_ssrs_t_last_4_clin',
    'c_ssrs_t_last_5_clin',

 #   'c_ssrs_t_2weeks_1_clin',
    'c_ssrs_t_2weeks_2_clin',
    'c_ssrs_t_2weeks_3_clin',
    'c_ssrs_t_2weeks_4_clin',
    'c_ssrs_t_2weeks_5_clin',

    'c_ssrs_1',
    'c_ssrs_2',
    'c_ssrs_3',
    'c_ssrs_4',
    'c_ssrs_5',

 #   'chameleon_ideation_stu'
]

In [3]:
times = {'intake': ['intake_arm_1', 'pre_treatment_arm_1'],
         'time2' : ['5th_session_arm_1', 'control_5weeks_arm_1'],
         'time3' : ['followup_3month_arm_1', 'control_3month_arm_1', 'control_6month_arm_1'],
        }
# if not os.path.exists("Liat graphs/plots"):
#     os.mkdir("Liat graphs/plots")

group_names = {
    '0': 'invalid',
    '1': 'ipt', # חירום
    '2' : 'tau', # רגיל
    '3': 'control' # מינימלית
}

# for time in times.keys():
# #     print(time)
#     create_dataset(event_names = times[time], path = f"Liat graphs/data/{time}/DeppClinic_patient_data1.csv")


In [4]:
df_times = {
    'intake': pd.read_csv(r'data/new_processing/intake/DeppClinic_patient_data.csv'),
    'time2' : pd.read_csv(r'data/new_processing/time2/DeppClinic_patient_data.csv'),
    'time3' : pd.read_csv(r'data/new_processing/time3/DeppClinic_patient_data.csv')
         }

intake_target_variables =  ['suicidal_ideation', 'suicidal_behavior',
        'suicidal_attempt', 'ER', 'NSSI']

time2_target_variables = ['suicidal_ideation', 'suicidal_behavior', 
        'suicidal_attempt', 'ER', 'Psychiatric', 
        'NSSI', 'finished_treatment']

target_variables = {
    'intake': intake_target_variables,
    'time2': time2_target_variables, 
    'time3': time2_target_variables}
        

In [5]:
for time in times:
    df = df_times[time]
    df_times[time] = df[df['age_child_pre'] < 12.5]

In [6]:
def rename_groups(df, group_names):
    df['group'] = '0'
    df.loc[df['group___1'] == 1, 'group'] = '1'
    df.loc[df['group___2'] == 1, 'group'] = '2'
    df.loc[df['group___3'] == 1, 'group'] = '3'
    df['group'] = df['group'].map(group_names)
    return df

for time in times:
    df = df_times[time]
    df_times[time] = rename_groups(df, group_names)
    df_times[time] = df_times[time][df_times[time]["group"].isin(["ipt", "control"])]

## Intake

In [7]:
df_times['intake'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  1.0                  31
         0.0                  12
ipt      1.0                  20
         0.0                   7
Name: count, dtype: int64

In [8]:
df_intake_ids = pd.read_csv(r"old_process_intake_ids.csv")
df_times['intake'] = df_times['intake'][df_times['intake'].id.isin(df_intake_ids.id)]

In [9]:
df_times['intake'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  1.0                  18
         0.0                  10
ipt      1.0                  18
         0.0                   5
Name: count, dtype: int64

In [10]:
df_times['intake'].query("group == 'control' & suicidal_ideation == 1").id.values

array(['E5541', 'H0740', 'S5556', 'Y7918', 'D7663', 'H1354', 'R5907',
       'T0543', 'I6535', 'S6442', 'E3613', 'T5527', 'I4694', 'R7617',
       'N6117', 'A1651', 'A3781', 'T4075'], dtype=object)

## Time 2

In [15]:
df_times['time2'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  0.0                  12
         1.0                  10
ipt      0.0                  15
         1.0                  15
Name: count, dtype: int64

In [16]:
time2_ids = pd.read_csv(r"old_process_time2_ids.csv")
df_times['time2'] = df_times['time2'][df_times['time2'].id.isin(time2_ids.id)]
df_times['time2'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  0.0                  12
         1.0                  10
ipt      0.0                  15
         1.0                  15
Name: count, dtype: int64

In [13]:
df_times['intake'][suicidal_ideation_time2 + ['suicidal_ideation', 'id', 'group']]

,c_ssrs_t_last_1_clin,c_ssrs_t_last_2_clin,c_ssrs_t_last_3_clin,c_ssrs_t_last_4_clin,c_ssrs_t_last_5_clin,c_ssrs_t_2weeks_2_clin,c_ssrs_t_2weeks_3_clin,c_ssrs_t_2weeks_4_clin,c_ssrs_t_2weeks_5_clin,c_ssrs_1,c_ssrs_2,c_ssrs_3,c_ssrs_4,c_ssrs_5,suicidal_ideation,id,group
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i6535,control
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N4125‏,ipt
15,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,E5541,control
21,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,N7245,ipt
24,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,H0740,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,A1651,control
511,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,T8560,ipt
515,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,A3781,control
522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N7753,control


In [14]:
df_times['time2']

,id,gender,redcap_event_name,age_child_pre,parents_born_m,parents_born_2_m,born_child_m,born_child_2_m,parent_religion_m,parent_religion_other_m,...,group___2,group___3,suicidal_ideation,suicidal_behavior,suicidal_attempt,ER,Psychiatric,NSSI,finished_treatment,group
0,S6442,2.0,5th_session_arm_1,10.5,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,control
1,E3613,1.0,5th_session_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,control
5,T5527,3.0,5th_session_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,control
10,D4440,NaN,5th_session_arm_1,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ipt
14,Y5064,2.0,5th_session_arm_1,8.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,ipt
15,APPD9667,1.0,5th_session_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,control
17,R7617,2.0,5th_session_arm_1,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,control
40,T4075,2.0,5th_session_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,control
49,E5541,NaN,5th_session_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,control
55,N7245,1.0,5th_session_arm_1,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,ipt


## Time 3

In [15]:
df_times['time3'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  0.0                  15
         1.0                  12
ipt      0.0                  18
         1.0                   9
Name: count, dtype: int64

In [17]:
time3_ids = pd.read_csv(r"old_process_time3_ids.csv")
df_times['time3'] = df_times['time3'][df_times['time3'].id.isin(time3_ids.id)]
df_times['time3'].groupby('group').suicidal_ideation.value_counts()

group    suicidal_ideation
control  0.0                  12
         1.0                   5
ipt      0.0                  17
         1.0                   6
Name: count, dtype: int64

In [ ]:
for target in target_variables['intake']:
    intake = df_times['intake']
    time2 = df_times['time2']
    
    intake = intake[['group', target]]
    time2 = time2[['group', target]]

    intake_values = intake.groupby('group')[target].sum()[['ipt', 'control']].values
    intake_gb = intake.groupby('group')[target].sum()[['ipt', 'control']]

    time2_values = time2.groupby('group')[target].sum()[['ipt', 'control']].values
    time2_gb = time2.groupby('group')[target].sum()[['ipt', 'control']]

    df = pd.DataFrame({'intake': intake_values, 'time2': time2_values})
    print(f"\n\n{target =}\n\n{intake_gb = }\n\n{time2_gb = }\n\n")
#     print('\n\n\n\n\n',target)
#     print(mcnemar(df.values, exact=False))

In [9]:
for target in target_variables['intake']:
    intake = df_times['intake']
    time3 = df_times['time3']
    
    intake = intake[['group', target]]
    time3 = time3[['group', target]]

    intake_values = intake.groupby('group')[target].sum()[['ipt', 'control']].values
    time3_values = time3.groupby('group')[target].sum()[['ipt', 'control']].values

    df = pd.DataFrame({'intake': intake_values, 'time3': time3_values})
    print('\n\n\n\n\n', target)
    print(mcnemar(df.values, exact=False))






 suicidal_ideation
pvalue      1.0
statistic   0.0





 suicidal_behavior
pvalue      0.16913144470266755
statistic   1.890625





 suicidal_attempt
pvalue      0.16913144470266755
statistic   1.890625





 ER
pvalue      0.10017829422626791
statistic   2.7027027027027026





 NSSI
pvalue      0.16913144470266755
statistic   1.890625
